# Utilities

In [ ]:
### Importing general modules ...

import random
import healpy as hp
import glob
from scipy.optimize import curve_fit
import pickle
from importlib import reload
import time
import scipy
import os
import numpy as np
import matplotlib.pyplot as plt
import sys
import pylab
from pylab import arange, show, cm
from astropy import units as uq

### Specific qubic modules
from qubicpack.utilities import Qubic_DataDir
from pysimulators import FitsArray
import pysm 
import pysm.units as u
import qubic
from qubic import QubicSkySim as qss
from qubic import fibtools as ft
from qubic import camb_interface as qc
from qubic import SpectroImLib as si
from qubic import NamasterLib as nam
from qubic import mcmc
from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator

### FGBuster functions module
from fgbuster import get_instrument, get_sky, get_observation  # Predefined instrumental and sky-creation configurations
from fgbuster.visualization import corner_norm
from fgbuster.mixingmatrix import MixingMatrix
from fgbuster.observation_helpers import _rj2cmb, _jysr2rj

# Imports needed for component separation
from fgbuster import (CMB, Dust, Synchrotron,  # sky-fitting model
                      basic_comp_sep)  # separation routine

reload(qss)
reload(ft)

plt.rc('figure', figsize=(16, 10))
plt.rc('font', size=15)
plt.rcParams['image.cmap'] = 'jet'

In [ ]:
### Some initializations, to be replaced with specific path, or to modify in bash
os.environ['QUBIC_DATADIR'] = '/Users/edgarjaber/myqubic/qubic'
os.environ['QUBIC_DICT'] = '/Users/edgarjaber/myqubic/qubic/dicts'
global_dir = Qubic_DataDir(datafile='instrument.py', datadir=os.environ['QUBIC_DATADIR'])

In [ ]:
### This is for the FI
config = 'FI-150'
### Read some stuff
    # Read dictionary chosen
dictfilename = global_dir + '/doc/FastSimulator/FastSimDemo_{}.dict'.format(config)
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)
d['nside']=256
    #Define the number of reconstruction bands:
nbands = 3
d['nf_recon'] = nbands

center = qubic.equ2gal(d['RA_center'], d['DEC_center'])

In [ ]:
### Calculate the coverage map
d['nf_sub'] = nbands
d['npointings'] = 3000
p = qubic.get_pointing(d)
s = qubic.QubicScene(d)
    # Qubic Instrument
q = qubic.QubicMultibandInstrument(d)
    # number of sub frequencies for reconstruction
_, nus_edge, _, _, _, _ = qubic.compute_freq(150, d['nf_sub'])
    # Operator for Maps Reconstruction
a = qubic.QubicMultibandAcquisition(q, p, s, d, nus_edge)
    # Coverage
coverage = a.get_coverage()
coverage = np.sum(coverage, axis=0)# Sum the bands
coverage /= np.max(coverage)# Normalize by the max
hp.mollview(coverage)

In [ ]:
#To release memory
a=0
seenpix = coverage != 0

In [ ]:
#Same resolution function
def same_resol_maps(freq_map_qubic, freq_map_pysm, FWHMs, coverage, d):
    
    """
    Function that returns QSS maps and PySm maps at the resolution of the lowest frequency of QSS
    
    input:
        freq_map_qubic, freq_map_pysm: np.array, corresponding maps
        FWHMs: list, of FWHMs for the convolution kernel
        coverage: np.array, coverage map
        d: dict, qubic dictionary
        
    returns:
        freq_map_qubic, freq_map_pysm: np.array at the good resolution 
    """
    
    def get_sigma_iband(q, iband):
    """Get the sigma of one band."""
        sigma = q[iband].synthbeam.peak150.sigma * (150e9 / q[iband].filter.nu)  
        return sigma

    def get_sigma_prime(q, iband):
    """Get the sigma needed to convolve one band at the first band resolution."""
        sigma_band0 = get_sigma_iband(q, 0)
        sigma_bandi = get_sigma_iband(q, iband)
        sigma_prime = np.sqrt(sigma_band0**2 - sigma_bandi**2)
        print(sigma_band0, sigma_bandi, sigma_prime)
        return sigma_prime
    
    seenpix = coverage != 0
    freq_map_pysm[:,:,~seenpix] = 0
    print('')
    print('Reconvolving PySM to QSS resolutions')
    for i in range(d['nf_recon']):
            myFWHM = np.deg2rad(FWHMs[i])
            print('Convolving PySM band {} with {} rad {} deg'.format(i, myFWHM, FWHMs[i]))
            freq_map_pysm[i,:,:] = hp.sphtfunc.smoothing(freq_map_pysm[i,:,:], fwhm=myFWHM,
                                                              verbose=False)
    freq_map_pysm[:,:,~seenpix] = 0 
    print('')
    print(np.std((freq_map_pysm - freq_map_qubic)[:,:,seenpix], axis=2))
    print('')
    print('Now convolving PySM and QSS to worst resolution')
    for i in range(d['nf_recon']):
            myFWHM = 2.35*get_sigma_prime(q,i)
            print('Convolving PySM band {} with {} rad {} deg'.format(i, myFWHM, np.rad2deg(myFWHM)))
            print(np.sqrt(FWHMs[i]**2 + np.rad2deg(myFWHM)**2))
            freq_map_pysm[i,:,:] = hp.sphtfunc.smoothing(freq_map_pysm[i,:,:], fwhm=myFWHM,
                                                              verbose=False) 
            print('Convolving QSS band {} with {} rad {} deg'.format(i, myFWHM, np.rad2deg(myFWHM)))
            freq_map_qubic[i,:,:] = hp.sphtfunc.smoothing(freq_map_qubic[i,:,:], fwhm=myFWHM,
                                                              verbose=False)
    freq_map_qubic[:,:,~seenpix] = 0   
    freq_map_pysm[:,:,~seenpix] = 0
    
    return(freq_map_qubic, freq_map_pysm)

In [ ]:
Stokes = ['I', 'Q', 'U']
def SED(q, coverage, d, pix):
    
    """
    Procedure that returns the amplitude value of a pixel as a function of frequency
    
    inputs:
        q: dict, qubic dict
        coverage: np.array, coverage map
        d: dict, qubic dictionary
        pix: list, index of the seen pixels
    
    returns:
        plots of the SEDs
    """
    
    global Stokes
    reload(qss)
    # Can be modified ... 
    sky_config_1 = {'dust': 'd0'}
    sky_config_2 = {'CMB':'c1'}
    Qubic_sky_1 = qss.Qubic_sky(sky_config_1, d)
    Qubic_sky_2 = qss.Qubic_sky(sky_config_2, d)
    _, qss_d0, _, _ = Qubic_sky_1.get_partial_sky_maps_withnoise(spatial_noise=False, coverage=coverage)
    _, qss_c1, _, _ = Qubic_sky_2.get_partial_sky_maps_withnoise(spatial_noise=False, coverage=coverage)
    FWHMs1 = Qubic_sky_1.dictionary['synthbeam_peak150_fwhm'] * 150. / Qubic_sky_1.qubic_central_nus * Qubic_sky_1.fi2td
    FWHMs2 = Qubic_sky_2.dictionary['synthbeam_peak150_fwhm'] * 150. / Qubic_sky_2.qubic_central_nus * Qubic_sky_2.fi2td
    
    print(FWHMs1)
    print(FWHMs2)
    
    qss_d0 = np.transpose(qss_d0, (0, 2, 1)) 
    qss_c1 = np.transpose(qss_c1, (0, 2, 1))

    instrument = get_instrument('qubic')
    pysm_d0 = get_observation(instrument, 'd0', nside=256)
    pysm_c1 = get_observation(instrument, 'c1', nside=256)

    qss_d0, pysm_d0 = same_resol_maps(qss_d0, pysm_d0, FWHMs1, coverage, d)
    qss_c1, pysm_c1 = same_resol_maps(qss_c1, pysm_c1, FWHMs2, coverage, d)
    

    X = instrument.frequency()
    fig, axs = plt.subplots(2, 3)
    for i in range(3):
        axs[0,i].plot(X, qss_d0[:,i,pix[10]], 'bo', label='d0 QSS ' + Stokes[i])
        axs[0,i].plot(X, pysm_d0[:,i,pix[10]], 'go', label='d0 PySM ' + Stokes[i])
        axs[0,i].set_xlabel("f")
        axs[0,i].legend()
        axs[1,i].plot(X, qss_c1[:,i,pix[10]], 'bo', label='c1 QSS ' + Stokes[i])
        axs[1,i].plot(X, pysm_c1[:,i,pix[10]], 'go',label='c1 PySM ' + Stokes[i])
        axs[1,i].set_xlabel("f")
        axs[1,i].legend()
    

# Applying parametric CS with FGBuster

## d0

In [ ]:
reload(qss)

sky_config = {'dust':'d0'}
Qubic_sky = qss.Qubic_sky(sky_config, d)

#Raw qss map
raw_qss_map = np.transpose(Qubic_sky.get_fullsky_convolved_maps(), (0, 2, 1))
#Raw pysm map (with bp)
raw_pysm_map = np.transpose(Qubic_sky.get_simple_sky_map(), (0, 2, 1))
instrument = get_instrument('qubic')


npix = 12 * d['nside'] ** 2
Nf = int(d['nf_sub'])
band = d['filter_nu'] / 1e9
filter_relative_bandwidth = d['filter_relative_bandwidth']
_, nus_edge, nus_in, _, _, Nbbands_in = qubic.compute_freq(band, Nf, filter_relative_bandwidth)

new_list_of_freqs = []

for i in range(Nf):
    nfreqinteg = 50
    freqs = np.linspace(nus_edge[i], nus_edge[i + 1], nfreqinteg)
    weights_flat = np.ones(nfreqinteg)
    weights = weights_flat / _jysr2rj(freqs)
    weights /= _rj2cmb(freqs)
    weights /= np.trapz(weights, freqs * 1e9)
    new_list_of_freqs.append((freqs, weights))

instrument.frequency = new_list_of_freqs
print(new_list_of_freqs)

In [ ]:
FWHMs = Qubic_sky.dictionary['synthbeam_peak150_fwhm'] * 150. / Qubic_sky.qubic_central_nus * Qubic_sky.fi2td
freq_map_qubic, freq_map_pysm = same_resol_maps(raw_qss_map, raw_pysm_map, FWHMs, coverage, d)